In [25]:
from django.db.models import Count
from django.db.models import Q

In [26]:
## total batch
Batch.objects.all().count()

2

In [27]:
## total images
Image.objects.all().count()

10

In [28]:
## count images that has any label
Image.objects.annotate(label_count=Count('label')).filter(label_count__gt=0).count()

7

In [29]:
## total images labelled by "labeller1"
Image.objects.annotate( label_count=Count('label', distinct=True) )\
    .filter(
        Q(label_count__gt=0) & 
        Q(batch__labeller__user__username="labeller1")
    ).count()

3

In [57]:
# list of total images labelled by all user
msg=""
for u in MyUser.objects.all().order_by('user__username'):
    n=Image.objects.annotate( label_count=Count('label', distinct=True) )\
    .filter(
        Q(label_count__gt=0) & 
        Q(batch__labeller__user__username=u.user.username)
    ).count()
    
    m=Image.objects.annotate( label_count=Count('label', distinct=True) )\
    .filter(
        Q(label_count__gt=0) & 
        Q(batch__labeller__user__username=u.user.username)
    ).count()
    
    msg+= "%15s: labelled %05d, reviewed %05d\n"%(u,n,m)
print msg

          admin: labelled 00000, reviewed 00000
      labeller1: labelled 00003, reviewed 00003
      labeller2: labelled 00004, reviewed 00004



In [61]:
# list of total images labelled by all user
msg=""
for u in MyUser.objects.all().order_by('user__username'):
    n=Image.objects.annotate( label_count=Count('label', distinct=True) )\
    .filter(
        Q(label_count__gt=0) & 
        Q(batch__labeller__user__username=u.user.username)
    ).count()
    
    m=Image.objects.annotate( label_count=Count('label', distinct=True) )\
    .filter(
        Q(label_count__gt=0) & 
        Q(batch__labeller__user__username=u.user.username)
    ).count()
    
    msg+= "+%15s labelled: %5d, reviewed: %5d\n"%(u,n,m)
print msg

+          admin labelled: 00000, reviewed: 00000
+      labeller1 labelled: 00003, reviewed: 00003
+      labeller2 labelled: 00004, reviewed: 00004



## Testing

In [6]:
for i in Image.objects.annotate(label_count=Count('label')).filter(Q(label_count__gt=0)&Q(batch__labeller__isnull=False)):
    print "%s did %s label(s) on %s"%(i.batch.labeller, i.label_count, i.raw_path)

labeller1 did 1 label(s) on 15185e10-3d59-4129-b5c4-314fdb228a59.jpg
labeller1 did 1 label(s) on 25136c78-05f6-422c-9b82-cbbd42deb261.jpg
labeller1 did 1 label(s) on 34c84071-abd7-4f01-86e6-3f2dc6c96a0b.jpg
labeller2 did 1 label(s) on 7696acee-37df-4d8c-b85b-30220ac00020.jpg
labeller2 did 1 label(s) on 88634d71-c69f-4582-b54c-926719da1020.jpg
labeller2 did 1 label(s) on 92506a5f-1f28-482a-98ad-e377c7ddfed3.jpg
labeller2 did 2 label(s) on bbd100a5-82e3-4bcd-8213-24d6ad73ffc6.jpg


In [7]:
Image.objects.annotate(label_count=Count('label', distinct=True))\
    .filter(Q(label_count__gt=0)&Q(batch__labeller__isnull=False))\
    .values("batch__labeller").annotate(job_count=Count('batch__labeller', distinct=True))

[{'job_count': 1, 'batch__labeller': 2}, {'job_count': 1, 'batch__labeller': 3}]

In [8]:
Batch.objects.all().values('labeller').annotate(total=Count('labeller')).order_by('total')

[{'total': 1, 'labeller': 2}, {'total': 1, 'labeller': 3}]

In [14]:
for i in MyUser.objects.annotate(review_done_count=Count('reviewer_batch')):
    print "%s reviewed %s batches"%(i,i.review_done_count)

admin reviewed 1 images
labeller1 reviewed 0 images
labeller2 reviewed 0 images


In [16]:
for i in MyUser.objects.annotate(review_done_count=Count('labeller_batch')):
    print "%s reviewed %s batches"%(i,i.review_done_count)

admin reviewed 0 batches
labeller1 reviewed 1 batches
labeller2 reviewed 1 batches


In [30]:
for i in MyUser.objects.annotate(x=Count('labeller_batch__image')):
    print "%s, %s"%(i,i.x)

admin, 0
labeller1, 5
labeller2, 5


In [40]:
MyUser.comment_set

[]